# Transfer Learning on google/vit-base-patch16-224
- From HuggingFace

In [1]:
!pip3 install --quiet evaluate transformers
!pip3 install --quiet 'accelerate>=0.26.0'

In [2]:
import io
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn

from huggingface_hub import notebook_login
from datasets import Dataset
from transformers import AutoImageProcessor, ViTForImageClassification
from datasets import Image as HFImage
from PIL import Image

from transformers import TrainingArguments, Trainer
import evaluate

## Load Dataset:

In [3]:
qr_images_path = r"data/qr_images/qr_images"
qr_labels_path = r"data/qr_labels.csv"

In [4]:
def get_all_qr_images(filepath: str) -> list:
    """
    Takes in the path to the directory holding all QR images, and return the image file paths as a list
    
    Args:
        filepath: The path to the directory holding the QR images
        
    Returns:
        list: A list holding full paths to QR image files
    """
    
    # Defining sorting helper function
    def extract_number(filename: str) -> int:
        """
        Extracts the ID or number corresponding to the QR image
        
        Args:
            filename: The QR image
            
        Returns:
            int: The ID or number corresponding to the QR image
        """
        image_number = int(filename.split("_")[1].split(".")[0])
        return image_number
    
    # Extracting all items from provided path
    all_items = os.listdir(filepath)
    
    # Keeping only the files from the list
    all_files = [f for f in all_items if os.path.isfile(os.path.join(filepath, f))]
    
    # Numerically sorting the files
    sorted_files = sorted(all_files, key=extract_number)
    
    # Return full paths so downstream code can open files correctly
    full_paths = [os.path.join(filepath, f) for f in sorted_files]
    return full_paths

In [5]:
y_labels = pd.read_csv(qr_labels_path, index_col=0)
labels = y_labels['label'].tolist()
labels[:5]

[1, 1, 1, 1, 1]

In [6]:
# Hugging Face Lazy List
image_paths = get_all_qr_images(qr_images_path)

dataset = Dataset.from_dict({"image": image_paths, "label": labels})
dataset = dataset.cast_column("image", HFImage())

dataset[255:265]['label']

[1, 0, 1, 1, 0, 0, 1, 1, 1, 1]

### Train Val Test Split

In [7]:
dataset_split = dataset.train_test_split(
    test_size=0.2,
    #stratify_by_column="label",
    seed=42
    )

train_dataset = dataset_split["train"]
test_dataset = dataset_split["test"]

## Preprocessing:

In [8]:
# Load image processor
""" 
Prepares input features for vision models and post processing their outputs by 
    transformations such as resizing, normalization, and conversion to PyTorch 
    and Numpy tensors.
"""
processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
processor

label2id = ... # Fill in with dataset

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [9]:
def transforms(batch):
    """
    This function prepares a batch of raw image data and labels so they can be fed into a model.
    """
    batch['image'] = [Image.open(io.BytesIO(x['bytes'])).convert('RGB') for x in batch['image']]
    inputs = processor(batch['image'],return_tensors='pt')
    inputs['labels']=[dataset[y]['label'] for y in batch['label']]
    return inputs


def collate_fn(batch):
    """
    This function takes a list of inputs dictionaries (from transforms) and stacks them into batch tensors.
    """
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

## Evaluation Metrics

In [10]:
"""
Compute evaluation metrics for model predictions using the Hugging Face
    `evaluate` library.
"""
accuracy = evaluate.load('accuracy')

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits,axis=1)
    score = accuracy.compute(predictions=predictions, references=labels)
    return score

## Model

In [11]:
# Model initialization
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels = 2,
    id2label = {0: "Phishing", 1: "Benign"},
    label2id = {"Phishing": 0, "Benign": 1}, 
    ignore_mismatched_sizes=True
)   

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Freezing layers

In [12]:
# Freeze 85,827,109 params
for name,p in model.named_parameters():
    if not name.startswith('classifier'):
        p.requires_grad = False

num_params = sum([p.numel() for p in model.parameters()])

trainable_params = sum([p.numel() for p in model.parameters() if p.requires_grad])

print(f"{num_params = :,} | {trainable_params = :,}")

num_params = 85,800,194 | trainable_params = 1,538


### Training

In [13]:
# Define training arguments and train
training_args = TrainingArguments(
    output_dir="./vit-base-qr-classifier",
    per_device_train_batch_size=16,
    #evaluation_strategy="epoch",
    #save_strategy="epoch",
    logging_steps=100,
    num_train_epochs=5,
    learning_rate=.0001,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=True,
    report_to='none',
    #load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor
)

trainer.train()

/var/folders/vj/rxr6zwfn3rl3x08myxqw9ftw0000gn/T/ipykernel_82434/2793516926.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-691fc589-0803e87c53e9b53b212679b4;bcdf4e29-b918-4d85-b7e5-352f1d5d087b)

Invalid username or password.

## Model Evaluation

In [ ]:
trainer.evaluate(...)